## Abstract

Despite the header being `## Abstract`, this section will render as a highlighted section titled *Summary*.

----

:::{.callout-note title="AI usage disclosure" collapse="true"}
This is a placeholder for the AI usage disclosure. Once all authors sign the AI code form on AirTable, SlackBot will message you an AI disclosure that you should place here.
:::

## Introduction

Once edited by you, this file will become your publication. Alternatively, if you already have a notebook written that you're trying to transform into a pub, replace this file with your own, but be sure to add the YAML front matter (the first cell) to your notebook.

## Learn more

Refer to [Demo](/examples/demo.html) to learn about possible syntax.

In [ ]:
from analysis.preprocess import *

adata = preprocess_bone_marrow()

In [ ]:
# --- Configuration ---
config = {
    "seed": 24,
    "pca_components": 50,
    "network_class": "deepBilinearNet", #"deepReLUNet", # "deepSiLUNet"
    "train_params": {
        "lr": 8e-4,
        "epochs": 96,
        "batch_size": 1024,
        "pumap_kwargs": {"non_parametric_embeddings": True}
    },
    "jacobian_batch_size": 40,
    "plot_params": {
        "n_features": 16,
        "stat": "mean"
    },
    "output_filename": "bone_marrow_features.csv"
}

In [ ]:
# Select cell types to analyze
config["plot_params"]["celltypes"] = adata.obs["cell_type"].value_counts().index[:5].tolist()

# Step 1: Prepare data
train_dataset, adata_mean_zero = prepare_data(adata, config["pca_components"])

# Step 2: Define and train model
# Note: `network` must be an instantiated model class
from analysis.utils import PUMAP, Datamodule, Model
from analysis.networks import deepReLUNet, deepSiLUNet, deepBilinearNet

network = deepBilinearNet() # Replace with your model instantiation
encoder = train_parametric_umap(network, train_dataset, config["train_params"])

# Step 3: Generate and plot embedding
embedding = generate_and_plot_embedding(encoder, adata, train_dataset)

# Step 4: Compute Jacobian in gene space
jacobxall = compute_gene_space_jacobian(encoder.encoder, adata, train_dataset, adata_mean_zero, config["jacobian_batch_size"])

# Step 5: Plot feature importances
class_genesorted = plot_feature_importance(adata, embedding, jacobxall, **config["plot_params"])

# Step 6: Export results
export_results(class_genesorted, config["output_filename"])